# Light up the world, precious nerds!

Why stumble around in the dark when you need to visit the bathroom at night?
Make the Pi turn on lights upon detecting motion in the dark.
The GPIO pins interface with sensors to give the Pi eyes and ears.
Let us see how as we work through the schematics.


## What is GPIO?

A maker project is a system with components that exchange messages with the Pi. Sensors convey messages about external conditions like temperature, light and pressure. An actuator, such as motor, receives a message to energize. The GPIO pins provide the interface for exchanging messages. 

In the old days, a taxi-meter had a flag to indicate whether a taxi was vacant or not. Once the taxi had a fare, the operator manually rotated the flag to indicate the taxi was not available any more. A GPIO pin has a similar function as the taxi-meter flag. It conveys a simple dichotomous message - high or low. ![Flag](images/taxi_meter_flag.jpg)

When a sensor conveys information to the Pi, it puts a "high" or "low" message on a GPIO pin. Thus a temperature sensor may indicate high or low temperature. Or a moisture sensor may indicate dry or wet soil. To recieve information, a GPIO pin must be in listening mode, i.e. "input".

Similarly, the Pi can energize an actuator via GPIO pin by putting a "high" or "low" message on a GPIO pin. GPIO "high" and "low" signals tell hardware to switch on and off. To send information, a GPIO pin must be in transmitting mode, i.e. "output". 

*A GPIO message is constructed from electrical voltage. Communication requires that the sender and receiver speak the same language. Let us see how electrical hardware and the Pi communicate in a common language using voltage, starting with the simplest case.*

## GPIO Output: Blink an LED

A Pi speaks "high" or "low" in electrical voltage. A voltage of 3.3 V is "high". A voltage of 0 V is "low". The 3.3 V on the pin is sufficient to power a small LED light. Wire up the circuit like so. ![BlinkLED](images/ozWiz_dancingLights_e_blinkLED.png)

The schematic shows a Raspberry Pi with a cartoon of the GPIO pins as they appear on the board. The pins are numbered 1 through 40 in the order of appearance. Some pins simple serve to provide an output voltage (3.3 V or 5 V) and a reference 0 V (Ground). A few GPIO pins haver additional specialized uses in communicating complex messages based on timing of "high" and "low" pulses. Pins also have a GPIO ID number that the Maker can use when refering to pin numbers in code, being careful to specify upfront which numbering scheme is in use.

Future schematics will not show the Raspberry Pi. The following simplification is sufficient. ![simpleSchematic](images/ozWiz_dancingLights_e_blinkLED_simple.png)

Note that:
- The pin no. is the GPIO ID. 
- By convention, red wire is the "live" feed and black wire is the return path.
- Other circuitry components are depicted with standard symbols where available or representative images.

For the **Pi**, what comes out is a GPIO "high" at pin GPIO24. For the **LED**, what goes in is the "high" from Pi at the anode (long leg) and what comes out is electrical path to the ground (GND) at the cathode (short leg).

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/rMdXbVPZ60w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

*A voltage of 3.3 V is enough to power on an LED. However, the power requirements of devices are often beyond what the Pi itself can supply. We need to add additional "glue" circuity to operate devices with independent power supply. In the next section we shall see a few different ways of doing that.* 

## Relay GPIO Signals to High-Voltage Devices

So far, we have operated an LED from the Pi using a GPIO pin. The GPIO "high" of 3.3 V was sufficient to power the LED which requires very little power to run. In practical applications, the GPIO pin is rarely used to supply a device directly. A more general approach is to supply a device's power requirement independently of the Pi and use GPIO logic to operate an on-off switch in the device's voltage feed to control it. This approach is versatile and opens up the possibility of controlling devices with power requirement beyond that which the Pi can supply, including home appliances (like a gas furnace or a washing machine) that draw power from the mains.

A relay represents realization of this principle. The simplest relays are of electro-mechanical design where an electromagnet closes electrical contacts against opposing spring action. Relays are a popular choice among makers for home automation projects and are available as a "hardware attached on top" (HAT) for the Pi. 

Electromechanical Relay | Pi Hat with Relays
- | -
![Relay Module](images/relayModule.jpg) | ![piHat](images/raspberryPi_relayHat.jpg)

With the addition of a relay, the circuit to operate our LED looks like so. Note the modification to operate a lamp supplied from the mains. Substitute the lamp for any household device supplied by the mains to explore the versatility of this approach. 

Relay to LED | Relay to Lamp
- | - 
![Relay to LED](images/ozWiz_dancingLights_e_relayLED.png) | ![Relay to Lamp](images/ozWiz_dancingLights_e_relayMains.png)
*Operate an LED using GPIO via relay.*  | *Modify the LED circuit to operate a lamp that draws power from the mains supply.*

What is going on here? Let's exampine what goes in and what comes out for each device.

For the **Pi**, what comes out is a GPIO "high" at pin GPIO24. This is the same for both LED and Lamp. For the **relay**, what goes in is the "high" from Pi at pin S (signal) and what comes out is that the two pins labeled "normally open" (NO) are shorted. For the **LED** or **Lamp**, what goes in at one terminal is the "live" voltage feed routed via the the NO terminals that are "spliced in" and what comes out at the other terminal is the electrical path to the sink. 

Note that:
1. The relay has similar power requirements as the Pi and draws from the Pi's source via pins VCC (positive terminal) and GND (negative terminal).
2. The return path for the lamp is the neutral of the mains and that for the LED is the ground.
3. The relay is "spliced in" the cable used to plug in the lamp to the electrical outlet.

*Relays make it possible to use a Pi to control electrical hardware with power requirements beyond the ability of the Pi to supply. However, the mechanical nature of operation produces wear and tear and contacts wear out from sparking. What if there was an alternative without moving parts? Next, we'll look at how MOSFETs are just the thing!*

### Use a MOSFET as switching element

A relay is an electrically-operated switch. Its low power-requirement lends it versatility. However, the moving parts within a relay pose an Achilles heel. This is where MOSFETs come in.

A metal-oxide-semiconductor field-effect-transistor (MOSFET) is a voltage-operated electrical switch. (More precisely, a MOSFET is a voltage-controlled resistor.) The business end of the MOSFET has three terminals: Gate, Source and Drain. They are shown labeled in the figures, in the schematic representation as well as a popular "through hole" package. 

MOSFET Schematic | MOSFET Package
- | -
![MOSFET symbol](images/OzWiz_MOSFETschematic.png) | ![MOSFET package](images/MOSFET_package.jpg)
Use the voltage on gate (G) to regulate the resistance between drain (D) and source (S) | Apply voltage at or above the specified threshold at the gate (left-most) to short the drain (middle) and source (right-most). 

When the voltage at the gate exceeds a threshold, the source and drain terminals are shorted. Otherwise, as long as the gate voltage is below threshold, the source and drain are open. Use the MOSFET to interface an electrical load with GPIO like so.

MOSFET Usage Pattern |  MOSFET Pin-Out for Through-Hole Package
- | -
![MOSFET](images/OzWiz_MOSFETcircuit.png) | ![MOSFET circuit](images/OzWiz_MOSFETcircuit_package.png)


For the **Pi**, what comes out is the "high" (or "low") at a GPIO pin. For the **MOSFET**, what goes in is the "high" (or "low") from Pi at the gate. What comes it is that the source and drain are shorted (or open). For the **load**, what goes in in at the positive terminal is the voltage feed from the power supply. What comes out at the negative terminal is the electrical path to the sink, with the MOSFET's source and drain terminals spliced in. 

It is worth nothing that the configuration shown is typical of a type "n-channel" (as opposed to p-channel) MOSFET. In this configuration, the load is connected to drain and the source is connected to sink. This is an important thumb-rule to remember when wiring MOSFET into the circuit. 

Note:
1. Threshold at gate - 3.3 Vor 5 V
2. Two types - n-channel p-channel
3. Multiple packages

*So far, we have an approach to interface Pi with electrical hardware powered by battery or mains to automate on-off switching. This approach is versatile and will work with most household appliances. But how to sense events to switch devices on-off in response to? For that, we need to learn about sensors and how to use them with the Pi.*

## Detect Motion in Darkness


### Voltage-Divider Circuit





References:
1. Get the inside scoop on an electromechanical relay here: https://www.youtube.com/watch?v=MJz2DRZ1w9E
2. Take the vibe up a notch with party lights using relays on a Pi HAT here: https://www.youtube.com/watch?v=bOGltcgiXiU

In [4]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/MJz2DRZ1w9E" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [5]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/GrvvkYTW_0k" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [6]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/t_hPrz7rs34" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>